In [107]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import re
import jieba
import numpy as np

from wordcloud import WordCloud
from collections import Counter
import seaborn as sns

In [108]:
df = pd.read_csv('../../clean_data/comments.csv')
user = pd.read_csv('../../clean_data/userinfo.csv')
sa = pd.read_csv('../../data/sa340.csv')

/var/folders/p9/cl5b6dld6ss75rd9r5_0snd80000gn/T/ipykernel_26989/2941857000.py:1: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../../clean_data/comments.csv')


In [109]:
city_counts = df.groupby('city').size()
city_counts_sorted = city_counts.sort_values(ascending=False)
values = list(city_counts_sorted)
threshold_index = int(len(values) * 0.2)
print(threshold_index, threshold_index /len(values))
THRESHOLD = values[threshold_index - 1] # index 从0开始
print('记录分界点：', THRESHOLD) 

60 0.19933554817275748
记录分界点： 2085


In [110]:
# 超过分界点的城市
cities_above = city_counts[city_counts >= THRESHOLD].index.tolist()
PART_1 = df[~df.city.isin(cities_above)]

# 对这些超过阈值的城市重新采样
temp = df[df['city'].isin(cities_above)]
PART_2 = temp.groupby('city').apply(lambda x: x.sample(n=THRESHOLD)).reset_index(drop=True)

/var/folders/p9/cl5b6dld6ss75rd9r5_0snd80000gn/T/ipykernel_26989/954044273.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  PART_2 = temp.groupby('city').apply(lambda x: x.sample(n=THRESHOLD)).reset_index(drop=True)


In [111]:
df_study = pd.concat([PART_1, PART_2]).reset_index(drop=True)

In [112]:
len(df_study)

239849

In [113]:
df_study.to_csv('../../clean_data/comments_study.csv', index=None)

In [114]:
# 将用户信息和评论进行匹配
user = pd.read_csv('../../clean_data/userinfo.csv')
user = user[user.userid.isin(df_study.userId)]

In [115]:
user.to_csv('../../clean_data/user_study.csv', index=None)